# Enhanced AI-Powered HR Assistant for Nestle

## Nestlé HR Policy Chatbot Overview

This Jupyter Notebook implements a conversational chatbot designed to answer user queries based on information contained within Nestlé's HR policy documents. It leverages several powerful technologies from the field of Natural Language Processing (NLP) and Large Language Models (LLMs) to achieve this:

1.  **Document Loading and Chunking:** The notebook begins by loading the HR policy document (in PDF format) and splitting it into smaller, manageable text chunks. This is crucial because LLMs have limitations on the amount of text they can process at once.

2.  **Text Embeddings:** Each text chunk is then converted into a numerical vector representation called an "embedding." These embeddings capture the semantic meaning of the text, allowing the system to understand the relationships between different pieces of information. OpenAI's embedding models are used for this purpose.

3.  **Vector Database:** The embeddings are stored in a vector database (Chroma), which allows for efficient similarity search. This means that when a user asks a question, the system can quickly find the most relevant text chunks from the HR policy.

4.  **Question Answering with LLM:** The most relevant text chunks are then passed to a large language model (OpenAI's GPT model) along with the user's question. The LLM uses this context to generate a coherent and informative answer.

5.  **User Interface:** Finally, a user-friendly interface is created using Gradio, allowing users to easily interact with the chatbot.

In summary, this notebook demonstrates a complete workflow for building a question-answering system over a PDF document using state-of-the-art NLP and LLM techniques. This approach can be generalized to other document types and domains, making it a valuable tool for information retrieval and knowledge management. The notebook is structured with Markdown explanations and code blocks separated by functionality, making it easy to follow and understand the implementation details.


---


## Usage and Setup Guide

### Prerequisites:
1. **Environment Setup:**
   - Python 3.8 or higher
   - Required packages installed
   - .env file configured

2. **Required Environment Variables:**
   ```
   OPENAI_API_KEY=your_api_key_here
   PDF_DOC_PATH=path_to_your_pdf_file
   ```

### Running the Application:
1. Ensure all cells are run in order
2. Wait for the Gradio interface to launch
3. Interface will be available at http://127.0.0.1:7860

### Features:
- Real-time question answering
- Conversation history tracking
- Error handling and recovery
- Rate limiting protection

### Best Practices:
- Keep API keys secure
- Monitor usage rates
- Regular logging review
- Backup document sources

### Troubleshooting:
- Check environment variables
- Verify PDF file accessibility
- Monitor API rate limits
- Review error logs

## Installation of Libraries

This code block uses `pip` to install the necessary Python libraries. These libraries are essential for the chatbot's functionality:

- `openai`: For interacting with OpenAI's API services and models
- `langchain`: Core framework for building applications with language models
- `langchain-community`: Community-maintained integrations and tools for LangChain
- `langchain-openai`: Official OpenAI integration package for LangChain
- `chromadb`: Vector database for efficient storage and retrieval of text embeddings
- `pypdf`: PDF document loading and processing utility
- `gradio`: Framework for creating user-friendly web interfaces
- `tiktoken`: OpenAI's tokenizer for accurate token counting and management
- `python-dotenv`: For secure environment variable management
- `psutil`: For system resource monitoring and management
- `pathlib`: For cross-platform file path handling

**Required Additional Packages:**
- Standard library packages: `logging`, `json`, `datetime`, `collections`
- System utilities: `platform`, `sys`

**Installation Command:**
```bash
pip install openai langchain langchain-community langchain-openai chromadb pypdf gradio tiktoken python-dotenv psutil

In [9]:
# Install necessary libraries (run this in your notebook environment if needed)
# !pip install openai langchain langchain-community langchain-openai chromadb pypdf gradio tiktoken python-dotenv psutil

# Enhanced AI-Powered HR Assistant for Nestle
## Core Imports and Error Handling Classes

This section establishes the foundation of our application with comprehensive imports and error handling. Let's examine each component:

### Import Section Breakdown:
- **Standard Library Imports:**
  - `os`: File and environment variable operations
  - `logging`: Application-wide logging system
  - `typing`: Type hints for better code documentation
  - `datetime`, `time`: Timestamp and performance tracking
  - `collections.defaultdict`: Used in rate limiting implementation
  - `json`: For error and metrics serialization

- **External Dependencies:**
  - `openai`: OpenAI API interface
  - `dotenv`: Secure environment variable management
  - `langchain` components: Document processing and AI operations
  - `gradio`: Web interface creation

### Logging Configuration:
- **Format Details:**
  - Timestamp: When the log entry occurred
  - Logger name: Source of the log entry
  - Log level: Severity of the entry
  - File and line: Precise location in code
  - Message: Actual log content

In [1]:
import os
import sys
import logging
import platform
import psutil
from typing import List, Dict, Any, Optional
from datetime import datetime
from collections import defaultdict
from functools import wraps
from time import time
from pathlib import Path
import json

# Core dependencies
import openai
from dotenv import load_dotenv

# Document processing
from langchain.document_loaders import PyPDFLoader, UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter

# LangChain components
from langchain_openai import OpenAI, OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA

# UI
import gradio as gr


# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - [%(filename)s:%(lineno)d] - %(message)s',
    handlers=[
        logging.FileHandler('chatbot.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

/opt/homebrew/anaconda3/envs/1720690009_20241228_v2/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Error Handling and Configuration Classes

This section defines our custom error hierarchy and configuration management. Let's examine each component:

### Custom Error Classes:
1. **ChatbotError (Base Class):**
   - Custom initialization with message and error code
   - Timestamp tracking for error occurrence
   - JSON serialization for logging
   - Error code system for categorization

2. **Specialized Error Classes:**
   - `DocumentLoadError`: Document-specific issues
   - `ModelError`: AI model operation failures
   - `ConfigurationError`: Setup and config problems

### Key Features:
- Every error includes:
  - Timestamp of occurrence
  - Error code for categorization
  - Detailed message
  - JSON serialization for logging
  - Error type identification

### Configuration Class Features:

1. **Config Class Structure:**
  - Uses class attributes for configuration settings
  - All settings are centralized in one location
  - Makes it easy to modify system-wide settings

2. **Configuration Parameters:**
  - `CHUNK_SIZE`: Controls the size of text segments (1000 characters)
  - `CHUNK_OVERLAP`: Amount of overlap between chunks (0 characters)
  - `MODEL_TEMPERATURE`: Controls AI response randomness (0 for consistent responses)
  - `EMBEDDING_MODEL`: Specifies which OpenAI embedding model to use

3. **get_settings Method:**
  - Returns all configuration as a dictionary
  - Filters out private attributes (starting with '_')
  - Useful for logging and debugging

4. **Default Configuration:**
   - Predefined settings with sensible defaults
   - Type-safe configuration values
   - Centralized configuration management

5. **Environment Integration:**
   - Loads settings from environment variables
   - Type conversion for numeric values
   - Configuration validation

6. **Access Methods:**
   - Safe configuration retrieval
   - Error handling for missing keys
   - Configuration state tracking

### Environment Loading:
1. **Variable Validation:**
   - Required variable checking
   - Type validation
   - Path existence verification

2. **Security Features:**
   - Secure API key handling
   - Path validation and sanitization
   - Error handling for missing/invalid values

### Metrics Tracking:
1. **Performance Metrics:**
   - Operation duration tracking
   - Error frequency monitoring
   - Statistical aggregation

2. **Error Tracking:**
   - Error type categorization
   - Timestamp recording
   - Error frequency analysis

### Key Components:
- Configuration versioning
- Load time tracking
- Type validation
- Secure variable handling
- Comprehensive error tracking
- Statistical analysis tools

In [2]:
class ChatbotError(Exception):
    """Base exception class for chatbot errors"""
    def __init__(self, message: str = None, error_code: str = None):
        self.message = message or "An unexpected error occurred in the chatbot"
        self.error_code = error_code or "CHATBOT_ERROR"
        self.timestamp = datetime.now()
        
        formatted_message = f"[{self.error_code}] {self.timestamp}: {self.message}"
        super().__init__(formatted_message)
        
    def to_dict(self) -> Dict[str, Any]:
        """Convert error details to dictionary for logging or API responses"""
        return {
            "error_code": self.error_code,
            "message": self.message,
            "timestamp": self.timestamp.isoformat(),
            "error_type": self.__class__.__name__
        }
    
    def log_error(self, log_level: int = logging.ERROR):
        """Log the error with specified logging level"""
        logger.log(log_level, json.dumps(self.to_dict(), indent=2))

class ConfigurationError(ChatbotError):
    """Raised when there are configuration-related issues"""
    def __init__(self, message: str = None, config_key: str = None):
        self.config_key = config_key
        error_message = f"Configuration error{f' for {config_key}' if config_key else ''}: {message}"
        super().__init__(
            message=error_message,
            error_code="CONFIG_ERROR"
        )
        
    def to_dict(self) -> Dict[str, Any]:
        """Add configuration-specific details to error dictionary"""
        error_dict = super().to_dict()
        error_dict["config_key"] = self.config_key
        return error_dict

class DocumentLoadError(ChatbotError):
    """Raised when there are issues loading documents"""
    def __init__(self, message: str = None, file_path: str = None):
        self.file_path = file_path
        error_message = f"Failed to load document{f' {file_path}' if file_path else ''}: {message}"
        super().__init__(
            message=error_message,
            error_code="DOC_LOAD_ERROR"
        )
        
    def to_dict(self) -> Dict[str, Any]:
        """Add document-specific details to error dictionary"""
        error_dict = super().to_dict()
        error_dict["file_path"] = self.file_path
        return error_dict

class ModelError(ChatbotError):
    """Raised when there are issues with the AI model"""
    def __init__(self, message: str = None, model_name: str = None, query: str = None):
        self.model_name = model_name
        self.query = query
        error_message = f"Model error{f' for {model_name}' if model_name else ''}: {message}"
        super().__init__(
            message=error_message,
            error_code="MODEL_ERROR"
        )
        
    def to_dict(self) -> Dict[str, Any]:
        """Add model-specific details to error dictionary"""
        error_dict = super().to_dict()
        error_dict.update({
            "model_name": self.model_name,
            "query": self.query if self.query else None
        })
        return error_dict
class Config:
    """Configuration settings for the chatbot"""
    CHUNK_SIZE = 1000
    CHUNK_OVERLAP = 0
    MODEL_TEMPERATURE = 0
    EMBEDDING_MODEL = "text-embedding-ada-002"
    
    @classmethod
    def get_settings(cls) -> Dict[str, Any]:
        """Returns all configuration settings as a dictionary"""
        return {k: v for k, v in cls.__dict__.items() 
                if not k.startswith('_')}

## Environment Setup and API Key Management

This section handles the secure loading and validation of environment variables. Let's examine the components:

### load_env Function:
- **Purpose:**
  - Safely loads environment variables from a .env file
  - Validates that all required variables are present
  - Returns API key and document path as a tuple

### Implementation Details:
- **load_dotenv:**
  - Loads environment variables from .env file
  - `verbose=True` enables logging of the loading process

- **Required Variables Dictionary:**
  - Maps environment variable names to human-readable descriptions
  - Makes error messages more user-friendly
  - Currently checks for:
    * OPENAI_API_KEY: Required for API authentication
    * PDF_DOC_PATH: Path to the document being processed

- **Validation Process:**
  - Iterates through required variables
  - Checks if each variable exists and has a value
  - Raises ValueError with specific message if any are missing

- **Logging:**
  - Success message logged when all variables are loaded
  - Helps with debugging environment setup issues

### Error Handling:
- Raises ValueError if required variables are missing
- Includes specific variable name in error message
- Helps quickly identify configuration issues

In [3]:
class Config:
    """Configuration settings for the chatbot"""
    
    # Default configuration values
    DEFAULT_CONFIG = {
        'CHUNK_SIZE': 1000,
        'CHUNK_OVERLAP': 0,
        'MODEL_TEMPERATURE': 0,
        'EMBEDDING_MODEL': "text-embedding-ada-002",
        'MAX_RETRIES': 3,
        'RATE_LIMIT_PER_SECOND': 5,
        'LOG_LEVEL': logging.INFO
    }
    
    def __init__(self):
        self._config = self.DEFAULT_CONFIG.copy()
        self._load_time = datetime.now()
        
    def load_from_env(self) -> None:
        """Load configuration from environment variables"""
        try:
            for key in self.DEFAULT_CONFIG:
                env_value = os.getenv(f'CHATBOT_{key}')
                if env_value:
                    # Convert string values to appropriate types
                    if isinstance(self.DEFAULT_CONFIG[key], int):
                        self._config[key] = int(env_value)
                    elif isinstance(self.DEFAULT_CONFIG[key], float):
                        self._config[key] = float(env_value)
                    else:
                        self._config[key] = env_value
                        
            logger.info("Configuration loaded successfully")
            
        except ValueError as e:
            raise ConfigurationError(
                message=f"Invalid configuration value: {str(e)}",
                config_key=key
            )
    
    def get(self, key: str) -> Any:
        """Get configuration value"""
        if key not in self._config:
            raise ConfigurationError(
                message=f"Configuration key not found: {key}",
                config_key=key
            )
        return self._config[key]
    
    def to_dict(self) -> Dict[str, Any]:
        """Convert configuration to dictionary"""
        return {
            'config': self._config,
            'load_time': self._load_time.isoformat(),
            'age_seconds': (datetime.now() - self._load_time).total_seconds()
        }

def load_env() -> tuple:
    """
    Load and validate environment variables
    Returns:
        tuple: (api_key, pdf_doc_path)
    Raises:
        ConfigurationError: If required environment variables are missing
    """
    try:
        load_dotenv(verbose=True)
        
        required_vars = {
            'OPENAI_API_KEY': 'OpenAI API key',
            'PDF_DOC_PATH': 'PDF document path'
        }
        
        env_vars = {}
        for var, description in required_vars.items():
            value = os.getenv(var)
            if not value:
                raise ConfigurationError(
                    message=f"Missing {description}",
                    config_key=var
                )
            env_vars[var] = value
            
        # Validate PDF path exists
        pdf_path = Path(env_vars['PDF_DOC_PATH'])
        if not pdf_path.exists():
            raise ConfigurationError(
                message=f"PDF file does not exist: {pdf_path}",
                config_key='PDF_DOC_PATH'
            )
        
        logger.info("Environment variables loaded successfully")
        return env_vars['OPENAI_API_KEY'], str(pdf_path)
        
    except Exception as e:
        if not isinstance(e, ChatbotError):
            e = ConfigurationError(
                message=f"Environment loading failed: {str(e)}"
            )
        e.log_error()
        raise

class MetricsTracker:
    """Track performance metrics for the chatbot"""
    
    def __init__(self):
        self.metrics = defaultdict(list)
        
    def record_duration(self, operation: str, duration: float):
        """Record duration of an operation"""
        self.metrics[f"{operation}_duration"].append(duration)
        logger.debug(f"{operation} took {duration:.2f} seconds")
    
    def record_error(self, error: ChatbotError):
        """Record an error"""
        self.metrics["errors"].append(error.to_dict())
        error.log_error()
    
    def get_statistics(self) -> Dict[str, Any]:
        """Get statistical summary of metrics"""
        stats = {}
        for metric, values in self.metrics.items():
            if metric.endswith('_duration'):
                if values:
                    stats[metric] = {
                        'mean': sum(values) / len(values),
                        'min': min(values),
                        'max': max(values),
                        'count': len(values)
                    }
            elif metric == "errors":
                stats["error_count"] = len(values)
                stats["error_types"] = defaultdict(int)
                for error in values:
                    stats["error_types"][error["error_type"]] += 1
                    
        return stats

# Global instances
config = Config()
metrics = MetricsTracker()

## Text Processing and Vector Store Creation

This section implements the text processing pipeline, converting documents into searchable vector embeddings. Let's break down the components:

### TextProcessor Class:
- **Purpose:**
  - Handles all text processing operations
  - Creates and manages embeddings
  - Implements caching for performance optimization

### Class Components:

1. **Initialization:**
   - Creates text splitter with configured chunk size and overlap
   - Initializes OpenAI embeddings client
   - Sets up caching dictionary for frequently accessed embeddings

2. **Text Splitting Configuration:**
   - Uses CharacterTextSplitter for consistent chunk sizes
   - Chunk size and overlap from Config class
   - Helps manage token limits for AI models

3. **Embedding Creation:**
   - Uses OpenAI's embedding model
   - Converts text chunks to numerical vectors
   - Enables semantic search capabilities

4. **Vector Store:**
   - Creates Chroma database from documents
   - Stores embeddings for efficient retrieval
   - Enables similarity search operations

### Error Handling:
- Load time measurement
- Page count logging
- Success/failure metrics
- Duration tracking


### Performance Features:
- Caching mechanism for repeated queries
- Batch processing of documents
- Efficient vector storage and retrieval

In [4]:
def load_document(file_path: str) -> List:
    """
    Load and process document based on file type with enhanced error handling
    
    Args:
        file_path (str): Path to the document
    Returns:
        List: Loaded document pages
    Raises:
        DocumentLoadError: If document loading fails
    """
    start_time = time()
    try:
        logger.info(f"Starting document load: {file_path}")
        file_path = Path(file_path)
        
        # Validate file existence and permissions
        if not file_path.exists():
            raise DocumentLoadError(
                message="File does not exist",
                file_path=str(file_path)
            )
        if not file_path.is_file():
            raise DocumentLoadError(
                message="Path is not a file",
                file_path=str(file_path)
            )
            
        # Select appropriate loader based on file extension
        if file_path.suffix.lower() == '.pdf':
            loader = PyPDFLoader(str(file_path))
        else:
            raise DocumentLoadError(
                message=f"Unsupported file type: {file_path.suffix}",
                file_path=str(file_path)
            )
        
        # Load document with retries
        max_retries = config.get('MAX_RETRIES')
        for attempt in range(max_retries):
            try:
                documents = loader.load()
                
                # Log success metrics
                duration = time() - start_time
                metrics.record_duration('document_load', duration)
                logger.info(
                    f"Successfully loaded document with {len(documents)} pages in {duration:.2f} seconds"
                )
                return documents
                
            except Exception as e:
                if attempt < max_retries - 1:
                    logger.warning(f"Retry {attempt + 1}/{max_retries} failed: {str(e)}")
                    continue
                raise DocumentLoadError(
                    message=f"Failed after {max_retries} attempts: {str(e)}",
                    file_path=str(file_path)
                )
                
    except Exception as e:
        if not isinstance(e, ChatbotError):
            e = DocumentLoadError(
                message=str(e),
                file_path=str(file_path)
            )
        metrics.record_error(e)
        raise

class TextProcessor:
    """Handles text splitting and embedding creation with enhanced error handling and metrics"""
    
    def __init__(self):
        try:
            self.text_splitter = CharacterTextSplitter(
                chunk_size=config.get('CHUNK_SIZE'),
                chunk_overlap=config.get('CHUNK_OVERLAP')
            )
            self.embeddings = OpenAIEmbeddings()
            self._cache = {}
            self._stats = defaultdict(int)
            logger.info("TextProcessor initialized successfully")
            
        except Exception as e:
            raise ConfigurationError(
                message=f"Failed to initialize TextProcessor: {str(e)}"
            )
    
    def _split_documents(self, documents: List) -> List:
        """
        Split documents into chunks with error handling
        
        Args:
            documents (List): List of documents to split
        Returns:
            List: Split text chunks
        """
        start_time = time()
        try:
            texts = self.text_splitter.split_documents(documents)
            duration = time() - start_time
            metrics.record_duration('text_splitting', duration)
            self._stats['total_chunks'] = len(texts)
            logger.info(f"Split documents into {len(texts)} chunks in {duration:.2f} seconds")
            return texts
            
        except Exception as e:
            error = ModelError(
                message=f"Failed to split documents: {str(e)}",
                model_name="text_splitter"
            )
            metrics.record_error(error)
            raise error
    
    def process_documents(self, documents: List) -> Chroma:
        """
        Process documents into embeddings and store in Chroma
        
        Args:
            documents (List): List of document pages
        Returns:
            Chroma: Vector store with processed documents
        """
        start_time = time()
        try:
            logger.info("Starting document processing")
            
            # Split documents
            texts = self._split_documents(documents)
            
            # Create vector store
            logger.info("Creating vector store")
            db = Chroma.from_documents(texts, self.embeddings)
            
            # Record success metrics
            duration = time() - start_time
            metrics.record_duration('document_processing', duration)
            self._stats['processing_time'] = duration
            
            logger.info(
                f"Successfully created vector store in {duration:.2f} seconds. "
                f"Stats: {json.dumps(self._stats, indent=2)}"
            )
            return db
            
        except Exception as e:
            if not isinstance(e, ChatbotError):
                e = ModelError(
                    message=f"Failed to process documents: {str(e)}",
                    model_name="document_processor"
                )
            metrics.record_error(e)
            raise e
            
    def get_stats(self) -> Dict[str, Any]:
        """Get processing statistics"""
        return {
            'stats': self._stats,
            'cache_size': len(self._cache),
            'embedding_model': config.get('EMBEDDING_MODEL')
        }

## Rate Limiting and Question-Answering System

This section implements the core QA functionality with rate limiting protection. Let's examine each component:

### Rate Limiting Decorator:
- **Purpose:**
  - Prevents API abuse
  - Manages resource utilization
  - Ensures fair system usage

- **Implementation Details:**
  - Uses defaultdict to track function calls
  - Maintains timestamp history for each function
  - Enforces maximum calls per second limit

- **Features:**
  - Rolling window implementation
  - Function-specific limiting
  - Automatic cleanup of old timestamps

### QASystem Class:
- **Purpose:**
  - Manages question-answering functionality
  - Handles interaction with OpenAI's API
  - Processes user queries

1. **Initialization:**
   - Model configuration
   - Retriever setup
   - History tracking
   - Performance monitoring

2. **Query Processing:**
   - Input validation
   - Context retrieval
   - Response generation
   - Performance tracking

3. **History Management:**
   - Query logging
   - Response tracking
   - Duration monitoring
   - Pattern analysis

### Class Components:

1. **Initialization:**
   - Sets up RetrievalQA chain
   - Configures OpenAI model parameters
   - Connects to vector store retriever

2. **Query Processing:**
   - Rate-limited query handling
   - Performance timing for each query
   - Comprehensive error handling

3. **Response Generation:**
   - Uses LangChain's chain mechanism
   - Retrieves relevant context
   - Generates coherent responses

### Performance Monitoring:
- Query execution time tracking
- Detailed logging of operations
- Error tracking and reporting

### Error Handling:
- Custom error types for different failures
- Detailed error messages
- Logging of all operations

In [5]:
def rate_limit(max_per_second: int):
    """
    Enhanced rate limiting decorator with metrics tracking
    
    Args:
        max_per_second (int): Maximum number of calls allowed per second
    """
    calls = defaultdict(list)
    
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            now = time()
            func_name = func.__name__
            
            # Clean old calls
            calls[func_name] = [c for c in calls[func_name] if c > now - 1]
            
            # Check rate limit
            if len(calls[func_name]) >= max_per_second:
                error = ChatbotError(
                    message=f"Rate limit of {max_per_second} calls per second exceeded for {func_name}",
                    error_code="RATE_LIMIT_ERROR"
                )
                metrics.record_error(error)
                raise error
                
            # Record call
            calls[func_name].append(now)
            
            # Execute function
            try:
                result = func(*args, **kwargs)
                metrics.record_duration(func_name, time() - now)
                return result
            except Exception as e:
                if not isinstance(e, ChatbotError):
                    e = ModelError(
                        message=str(e),
                        model_name=func_name
                    )
                metrics.record_error(e)
                raise e
                
        return wrapper
    return decorator

class QASystem:
    """Enhanced question-answering system with metrics and error handling"""
    
    def __init__(self, vector_store: Chroma):
        """Initialize QA system with vector store"""
        try:
            self.qa_chain = RetrievalQA.from_chain_type(
                llm=OpenAI(temperature=config.get('MODEL_TEMPERATURE')),
                chain_type="stuff",
                retriever=vector_store.as_retriever()
            )
            self.query_history = []
            logger.info("QA system initialized successfully")
            
        except Exception as e:
            error = ModelError(
                message=f"Failed to initialize QA system: {str(e)}",
                model_name="QASystem"
            )
            metrics.record_error(error)
            raise error
    
    @rate_limit(max_per_second=5)
    def get_response(self, query: str) -> str:
        """
        Get response for user query with enhanced error handling
        
        Args:
            query (str): User's question
        Returns:
            str: Generated response
        """
        start_time = time()
        try:
            # Validate query
            if not query or not query.strip():
                raise ModelError(
                    message="Empty query provided",
                    query=query
                )
            
            # Process query
            response = self.qa_chain.invoke({"query": query})
            
            # Record successful query
            duration = time() - start_time
            self.query_history.append({
                'query': query,
                'timestamp': datetime.now().isoformat(),
                'duration': duration
            })
            
            logger.info(f"Query processed successfully in {duration:.2f} seconds")
            return response['result'] if isinstance(response, dict) else response
            
        except Exception as e:
            if not isinstance(e, ChatbotError):
                e = ModelError(
                    message=f"Failed to process query: {str(e)}",
                    model_name="QASystem",
                    query=query
                )
            metrics.record_error(e)
            raise e
    
    def get_stats(self) -> Dict[str, Any]:
        """Get QA system statistics"""
        return {
            'total_queries': len(self.query_history),
            'average_duration': sum(q['duration'] for q in self.query_history) / len(self.query_history) if self.query_history else 0,
            'query_history': self.query_history[-10:]  # Last 10 queries
        }


## User Interface Implementation with Gradio

This section creates an interactive web interface using Gradio. Let's examine the components in detail:

### ChatInterface Class:
- **Purpose:**
  - Creates and manages the web-based user interface
  - Handles conversation state management
  - Processes user interactions

### Class Components:

1. **Initialization:**
   - Stores reference to QA system
   - Initializes empty conversation history list
   - Sets up interface state management

2. **Response Handler (respond method):**
   - **Functionality:**
     * Processes incoming user messages
     * Manages conversation history
     * Handles error states
   
   - **Parameters:**
     * message: The user's input text
     * history: Current conversation history
   
   - **Return Values:**
     * Tuple containing (response, updated_history)
     * Formats error messages for display if needed

3. **Interface Creation (create_interface method):**
   - **Components:**
     * Chatbot display area for conversation history
     * Text input box for user questions
     * Clear button for resetting conversation
   
   - **Event Handlers:**
     * Submit: Processes user input and updates display
     * Clear: Resets conversation state
   
   - **Layout:**
     * Clean, intuitive design
     * Clear labels and placeholders
     * Responsive interface elements

### Error Handling:
- Graceful error message display
- Maintains conversation state during errors
- Prevents interface crashes

In [6]:
class ChatInterface:
    """Enhanced chat interface with error handling and metrics"""
    
    def __init__(self, qa_system: QASystem):
        self.qa_system = qa_system
        self.conversation_history = []
        self.interface_metrics = {
            'total_interactions': 0,
            'error_count': 0,
            'start_time': datetime.now().isoformat()
        }
    
    def respond(self, message: str, history: List) -> tuple:
        """
        Process user message with enhanced error handling
        
        Args:
            message (str): User's message
            history (List): Conversation history
        Returns:
            tuple: (message, updated_history)
        """
        self.interface_metrics['total_interactions'] += 1
        
        try:
            response = self.qa_system.get_response(message)
            self.conversation_history.append({
                'user_message': message,
                'bot_response': response,
                'timestamp': datetime.now().isoformat()
            })
            return response, history + [[message, response]]
            
        except Exception as e:
            self.interface_metrics['error_count'] += 1
            error_message = f"Error: {str(e)}"
            if not isinstance(e, ChatbotError):
                metrics.record_error(ChatbotError(error_message))
            return error_message, history + [[message, error_message]]
    
    def create_interface(self) -> gr.Blocks:
        """Create enhanced Gradio interface"""
        with gr.Blocks() as demo:
            # Header
            gr.Markdown("# Nestlé HR Chatbot")
            
            # Chat interface
            chatbot = gr.Chatbot()
            msg = gr.Textbox(
                label="Ask about Nestle's HR policies",
                placeholder="Type your question here...",
                show_label=True
            )
            
            # Control buttons
            with gr.Row():
                clear = gr.Button("Clear Conversation")
                show_stats = gr.Button("Show Statistics")
            
            # Statistics display
            stats_output = gr.JSON(label="System Statistics", visible=False)
            
            # Event handlers
            msg.submit(
                self.respond,
                [msg, chatbot],
                [msg, chatbot]
            )
            
            clear.click(
                lambda: (None, None),
                None,
                [msg, chatbot],
                queue=False
            )
            
            show_stats.click(
                lambda: {
                    'interface': self.interface_metrics,
                    'qa_system': self.qa_system.get_stats(),
                    'overall_metrics': metrics.get_statistics()
                },
                None,
                stats_output
            )
            
        return demo

## Main Application Execution

This section orchestrates the entire application startup and execution process. Let's break down the implementation:

### Main Function Components:

1. **Application Setup:**
   - **Environment Initialization:**
     * Loads environment variables
     * Validates API keys and paths
     * Sets up logging system
   
   - **Document Processing:**
     * Loads specified document
     * Processes text into manageable chunks
     * Creates vector embeddings

2. **System Initialization:**
   - **Vector Store:**
     * Creates Chroma database
     * Indexes document embeddings
     * Prepares for efficient retrieval
   
   - **QA System:**
     * Initializes question-answering chain
     * Connects to OpenAI API
     * Sets up rate limiting

3. **Interface Launch:**
   - Creates Gradio interface
   - Configures web server
   - Launches interactive interface

### Error Handling:
- **Comprehensive error catching:**
  * Environment setup errors
  * Document processing failures
  * Model initialization issues
  * Interface creation problems

- **Logging:**
  * Detailed error messages
  * Application state logging
  * Startup process tracking

### Execution Flow:
1. Loads environment configuration
2. Processes input documents
3. Initializes AI components
4. Creates and launches interface

In [7]:
class ApplicationState:
    """Manages global application state and initialization"""
    
    def __init__(self):
        self.startup_time = datetime.now()
        self.initialized = False
        self.components = {}
        self.health_status = {
            'status': 'initializing',
            'last_check': self.startup_time.isoformat()
        }
    
    def update_health(self, status: str, message: str = None):
        """Update application health status"""
        self.health_status.update({
            'status': status,
            'last_check': datetime.now().isoformat(),
            'message': message
        })
        logger.info(f"Health status updated: {status} - {message}")
    
    def get_status(self) -> Dict[str, Any]:
        """Get complete application status"""
        return {
            'health': self.health_status,
            'uptime_seconds': (datetime.now() - self.startup_time).total_seconds(),
            'initialized': self.initialized,
            'components': list(self.components.keys()),
            'config': config.to_dict(),
            'metrics': metrics.get_statistics()
        }

def initialize_application() -> ApplicationState:
    """
    Initialize all application components with comprehensive error handling
    
    Returns:
        ApplicationState: Application state manager
    """
    app_state = ApplicationState()
    
    try:
        logger.info("Starting application initialization")
        
        # Load environment variables and configuration
        logger.info("Loading environment and configuration")
        api_key, pdf_path = load_env()
        config.load_from_env()
        app_state.components['config'] = config
        
        # Load and process documents
        logger.info("Loading documents")
        documents = load_document(pdf_path)
        app_state.components['documents'] = f"Loaded {len(documents)} pages"
        
        # Initialize text processor
        logger.info("Initializing text processor")
        processor = TextProcessor()
        vector_store = processor.process_documents(documents)
        app_state.components['text_processor'] = processor
        app_state.components['vector_store'] = vector_store
        
        # Initialize QA system
        logger.info("Initializing QA system")
        qa_system = QASystem(vector_store)
        app_state.components['qa_system'] = qa_system
        
        # Initialize chat interface
        logger.info("Setting up chat interface")
        interface = ChatInterface(qa_system)
        app_state.components['interface'] = interface
        
        # Mark initialization as complete
        app_state.initialized = True
        app_state.update_health('healthy', 'Application initialized successfully')
        
        logger.info("Application initialization completed successfully")
        return app_state
        
    except Exception as e:
        error_message = f"Application initialization failed: {str(e)}"
        logger.error(error_message)
        app_state.update_health('error', error_message)
        
        if not isinstance(e, ChatbotError):
            e = ChatbotError(error_message)
        metrics.record_error(e)
        raise

def cleanup_resources():
    """Cleanup application resources"""
    try:
        logger.info("Cleaning up application resources")
        # Add cleanup logic here (e.g., closing file handles, database connections)
        
    except Exception as e:
        logger.error(f"Error during cleanup: {str(e)}")
        
    finally:
        logger.info("Application shutdown complete")

def main():
    """Main application entry point with enhanced error handling and monitoring"""
    app_state = None
    
    try:
        # Initialize application
        app_state = initialize_application()
        
        # Create and launch interface
        demo = app_state.components['interface'].create_interface()
        
        # Configure shutdown handling
        def handle_shutdown():
            logger.info("Shutdown initiated")
            cleanup_resources()
        
        # Launch interface with shutdown handling
        demo.launch(
            share=False,  # Set to True to create a public URL
            prevent_thread_lock=True
        )
        
        # Log successful startup
        logger.info(
            "Application started successfully\n" +
            json.dumps(app_state.get_status(), indent=2)
        )
        
    except Exception as e:
        error_message = f"Application startup failed: {str(e)}"
        logger.error(error_message)
        
        if app_state:
            app_state.update_health('error', error_message)
        
        if not isinstance(e, ChatbotError):
            e = ChatbotError(error_message)
        metrics.record_error(e)
        
        raise
        
    finally:
        cleanup_resources()

if __name__ == "__main__":
    try:
        main()
    except Exception as e:
        logger.critical(f"Fatal error: {str(e)}")
        sys.exit(1)

2024-12-29 17:44:45,709 - __main__ - INFO - [2334330675.py:43] - Starting application initialization
2024-12-29 17:44:45,710 - __main__ - INFO - [2334330675.py:46] - Loading environment and configuration
2024-12-29 17:44:45,713 - __main__ - INFO - [465277660.py:92] - Environment variables loaded successfully
2024-12-29 17:44:45,714 - __main__ - INFO - [465277660.py:33] - Configuration loaded successfully
2024-12-29 17:44:45,715 - __main__ - INFO - [2334330675.py:52] - Loading documents
2024-12-29 17:44:45,715 - __main__ - INFO - [5827190.py:14] - Starting document load: dataset/the_nestle_hr_policy_pdf_2012.pdf
2024-12-29 17:44:46,152 - __main__ - INFO - [5827190.py:47] - Successfully loaded document with 8 pages in 0.44 seconds
2024-12-29 17:44:46,153 - __main__ - INFO - [2334330675.py:57] - Initializing text processor
2024-12-29 17:44:46,164 - __main__ - INFO - [5827190.py:82] - TextProcessor initialized successfully
2024-12-29 17:44:46,165 - __main__ - INFO - [5827190.py:126] - Star

* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


2024-12-29 17:44:52,125 - __main__ - INFO - [2334330675.py:125] - Application started successfully
{
  "health": {
    "status": "healthy",
    "last_check": "2024-12-29T17:44:51.515899",
    "message": "Application initialized successfully"
  },
  "uptime_seconds": 6.415666,
  "initialized": true,
  "components": [
    "config",
    "documents",
    "text_processor",
    "vector_store",
    "qa_system",
    "interface"
  ],
  "config": {
    "config": {
      "CHUNK_SIZE": 1000,
      "CHUNK_OVERLAP": 0,
      "MODEL_TEMPERATURE": 0,
      "EMBEDDING_MODEL": "text-embedding-ada-002",
      "MAX_RETRIES": 3,
      "RATE_LIMIT_PER_SECOND": 5,
      "LOG_LEVEL": 20
    },
    "load_time": "2024-12-29T17:43:49.946347",
    "age_seconds": 62.178805
  },
  "metrics": {
    "document_load_duration": {
      "mean": 0.4368932247161865,
      "min": 0.4368932247161865,
      "max": 0.4368932247161865,
      "count": 1
    },
    "text_splitting_duration": {
      "mean": 0.00013685226440429688

2024-12-29 17:46:33,856 - httpx - INFO - [_client.py:1025] - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-29 17:46:34,696 - httpx - INFO - [_client.py:1025] - HTTP Request: POST https://api.openai.com/v1/completions "HTTP/1.1 200 OK"
2024-12-29 17:46:34,715 - __main__ - INFO - [3138667305.py:100] - Query processed successfully in 1.54 seconds


## Testing and System Reporting Utilities

This section implements comprehensive testing and system reporting capabilities for application monitoring and maintenance.

### Test Runner Implementation:
1. **Test Categories:**
   - Environment Validation:
     * API key verification
     * Path validation
     * Configuration checks
     * Permission testing
   
   - Document Processing:
     * Loading verification
     * Processing validation
     * Format handling
     * Error catching
   
   - QA System Testing:
     * Query processing
     * Response validation
     * Performance metrics
     * Error handling

2. **Test Flow Management:**
   - Sequential execution
   - Dependency handling
   - Error capture
   - Result reporting

3. **Performance Testing:**
   - Response timing
   - Resource usage
   - Load handling
   - Bottleneck identification

### System Reporting Features:
1. **Report Components:**
   - System Configuration:
     * Current settings
     * Environment state
     * Component status
     * Resource allocation
   
   - Performance Metrics:
     * Operation timings
     * Resource usage
     * Error rates
     * Response times
   
   - Error Analysis:
     * Error patterns
     * Frequency analysis
     * Impact assessment
     * Resolution tracking

2. **Report Generation:**
   - Real-time updates
   - Historical tracking
   - Trend analysis
   - Alert generation

3. **Monitoring Integration:**
   - Log aggregation
   - Metric collection
   - Alert triggering
   - Status tracking

In [9]:
def run_tests():
    """Basic application tests"""
    try:
        logger.info("Starting application tests")
        
        # Test environment loading
        api_key, pdf_path = load_env()
        assert api_key, "API key not loaded"
        assert pdf_path, "PDF path not loaded"
        
        # Test document loading
        documents = load_document(pdf_path)
        assert documents, "Documents not loaded"
        
        # Test text processing
        processor = TextProcessor()
        vector_store = processor.process_documents(documents)
        assert vector_store, "Vector store not created"
        
        # Test QA system
        qa_system = QASystem(vector_store)
        test_query = "What are the working hours?"
        response = qa_system.get_response(test_query)
        assert response, "No response received"
        
        logger.info("All tests passed successfully")
        return True
        
    except Exception as e:
        logger.error(f"Test failed: {str(e)}")
        return False

def generate_system_report() -> Dict[str, Any]:
    """Generate comprehensive system report"""
    return {
        'timestamp': datetime.now().isoformat(),
        'configuration': config.to_dict(),
        'metrics': metrics.get_statistics(),
        'environment': {
            'python_version': sys.version,
            'platform': platform.platform(),
            'memory_usage': psutil.Process().memory_info().rss / 1024 / 1024  # MB
        },
        'log_summary': {
            'error_count': len([r for r in metrics.metrics['errors']]),
            'last_errors': metrics.metrics['errors'][-5:]  # Last 5 errors
        }
    }

In [11]:
run_tests()
generate_system_report()

2024-12-29 17:48:53,527 - __main__ - INFO - [2109332838.py:4] - Starting application tests
2024-12-29 17:48:53,531 - __main__ - INFO - [465277660.py:92] - Environment variables loaded successfully
2024-12-29 17:48:53,532 - __main__ - INFO - [5827190.py:14] - Starting document load: dataset/the_nestle_hr_policy_pdf_2012.pdf
2024-12-29 17:48:53,703 - __main__ - INFO - [5827190.py:47] - Successfully loaded document with 8 pages in 0.17 seconds
2024-12-29 17:48:53,714 - __main__ - INFO - [5827190.py:82] - TextProcessor initialized successfully
2024-12-29 17:48:53,714 - __main__ - INFO - [5827190.py:126] - Starting document processing
2024-12-29 17:48:53,715 - __main__ - INFO - [5827190.py:104] - Split documents into 7 chunks in 0.00 seconds
2024-12-29 17:48:53,715 - __main__ - INFO - [5827190.py:132] - Creating vector store
2024-12-29 17:48:54,116 - httpx - INFO - [_client.py:1025] - HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
2024-12-29 17:48:54,188 - __main_

{'timestamp': '2024-12-29T17:48:55.339909',
 'configuration': {'config': {'CHUNK_SIZE': 1000,
   'CHUNK_OVERLAP': 0,
   'MODEL_TEMPERATURE': 0,
   'EMBEDDING_MODEL': 'text-embedding-ada-002',
   'MAX_RETRIES': 3,
   'RATE_LIMIT_PER_SECOND': 5,
   'LOG_LEVEL': 20},
  'load_time': '2024-12-29T17:43:49.946347',
  'age_seconds': 305.393591},
 'metrics': {'document_load_duration': {'mean': 0.2923696041107178,
   'min': 0.1708228588104248,
   'max': 0.4368932247161865,
   'count': 3},
  'text_splitting_duration': {'mean': 0.00011658668518066406,
   'min': 8.487701416015625e-05,
   'max': 0.00013685226440429688,
   'count': 3},
  'document_processing_duration': {'mean': 2.147731383641561,
   'min': 0.47304606437683105,
   'max': 5.327102899551392,
   'count': 3},
  'get_response_duration': {'mean': 1.4640367825826008,
   'min': 1.1291050910949707,
   'max': 1.7221591472625732,
   'count': 3}},
 'environment': {'python_version': '3.10.12 | packaged by conda-forge | (main, Jun 23 2023, 22:41:52